# Playground

A place to test the feature discovery pipeline!

In [1]:
from utils import *
from openai import OpenAI
import json

with open('keys.json') as f:
    keys = json.load(f)

gpt_client = OpenAI(api_key=keys['openai'])

In [7]:
GOODFIRE_API_KEY = 'sk-goodfire-tgwKZ-aupqofjOr1yMXrnALCT_CM86SpJkR12BGgYka5shI-35FYSA'# os.environ.get('GOODFIRE_API_KEY')
client = goodfire.Client(GOODFIRE_API_KEY)
TARGET_BEHAVIOR = "Talk about cats."
# PROMPT = "Which one is bigger, 9.9 or 9.11?"
PROMPT = "Tell me a story."
# "A train travels 120 km at 60 km/h, then 80 km at 40 km/h. What's the average speed?" #"Which one is bigger, 9.9 or 9.11?" #for now fixed

retriever = Retriever(client, "meta-llama/Meta-Llama-3-8B-Instruct")
scorer = Scorer(gpt_client, "gpt-4o-mini", scale=2.0)
judge = Judge(gpt_client, "gpt-4o-mini")
steered_model = SteeredModel(client, "meta-llama/Meta-Llama-3-8B-Instruct")
model_output = steered_model.generate(PROMPT)
print(f"{PROMPT=}")
print(f"{model_output=}")

critique = judge.judge_output(TARGET_BEHAVIOR, model_output, PROMPT)
print(f"{critique=}")
eval_score = parseEvalScore(critique)
print(f"================{eval_score=}")
with open("scorer_logs.txt", "a", encoding="utf-8") as f:
    f.write(f"{PROMPT=}")
    f.write(f"{model_output=}")
    f.write(f"{critique=}")
    f.write(f"\n\n=== eval = {eval_score}\n")
    
if eval_score > 7:
    print("Already good enough")
else:
    features = retriever.retrieve_features(TARGET_BEHAVIOR, k=5)
    scores = scorer.score_features(TARGET_BEHAVIOR, critique, features, [])
    steered_model.set_features(features, scores)
    model_output = steered_model.generate(PROMPT)
    for i in range(10):
        print(f"-----Epoch {i}-----")
        critique = judge.judge_output(TARGET_BEHAVIOR, model_output, PROMPT)
        scores = scorer.score_features(TARGET_BEHAVIOR, critique, features, scores)
        steered_model.set_features(features, scores)
        model_output = steered_model.generate(PROMPT)

        # parse critique to get the score from "Score: 1. The response does not meet the target behavior of focusing on cats."
        # and then stop if the score is good enough
        try:
            eval_score = float(re.findall(r'Score: (-?\d*\.?\d+)', critique)[0]);
        except:
            eval_score = 3
        print(f"================{eval_score=}")
        with open("scorer_logs.txt", "a", encoding="utf-8") as f:
            f.write(f"-----Epoch {i}-----")
            f.write(f"{model_output=}")
            f.write(f"{critique=}")
            f.write(f"\n\n=== eval = {eval_score}\n")
            
        if eval_score > 7:
            break
        
        print(f"{model_output=}")
        print(f"{critique=}")

PROMPT='Tell me a story.'
model_output="I'd love to! Let me spin you a yarn about a magical kingdom hidden deep in the heart of a dense forest. Once upon a time, in this enchanted land, there lived a kind and gentle princess named Sophia. She had hair as golden as the sun and eyes as blue as the clearest sky.\n\nSophia had a special gift – she could communicate with animals. She could understand their thoughts and feelings, and they could sense her kindness and compassion. The creatures of the forest adored Sophia, and she spent her days exploring the woods, helping those in need, and learning about the magic that dwelled within.\n\nOne day, a wise old owl named Hoot approached Sophia with a grave concern. A dark sorcerer had cast a spell on the forest, causing the trees to wither and the rivers to dry up. The animals were struggling to survive, and Sophia knew she had to act fast.\n\nWith Hoot by her side, Sophia set out on a quest to find the the"
critique='Score: 2. The response doe

In [11]:
critique.choices[0].message.content

'Critic:\n\nScore: 3. The response does not meet the target behavior of solving the math problem accurately. While the response attempts to reason step by step, there is a fundamental error in the approach, leading to an incorrect conclusion.\n\nKey Observations:\n- The crucial misunderstanding here is assuming that drying time scales linearly with the number of shirts, which is not typical in problems of this nature. If shirts are dried simultaneously, drying 30 shirts should also take approximately 1 hour, assuming they are under the same conditions.\n- There is a calculation error in converting hours to minutes and back that leads to an incorrect total in terms of drying time.\n\nSuggestions for Improvement:\n- Recognize that in a practical drying scenario, assuming all shirts have the same exposure, the drying time for 30 shirts should remain approximately 1 hour as long as they fit in the same space under identical conditions.\n- Correct the understanding of proportional relations

In [ ]:
[{'role': 'system', 'content': 'Instructions for the Scorer:\n\nYou are responsible for adjusting the importance of language features to guide the generation of messages that align with a given target behavior. Your output should be a list of scores corresponding to the available features, where:\n\t•\tScores range from -1 to 1:\n\t•\t-1: The feature is completely suppressed.\n\t•\t1: The feature is maximally boosted.\n\t•\tScores closer to 0 indicate a neutral impact on the feature.\n\nInput Information:\n\n\t1.\tTarget Behavior: Describes the overall intent or theme that the generated message should follow.\n\t2.\tAvailable Features: A list of language features (e.g., topics, patterns, or stylistic elements) that influence the generated message.\n\t3.\t(Optional) Previous Scores: Scores from prior adjustments that indicate the importance of features before this iteration.\n\t4.\t(Optional) Critique from a Judge: Feedback on how the message generated using the previous scores deviated from the target behavior. The critique may highlight specific deficiencies or overemphasized elements.\n\nUsing the provided inputs, adjust the feature scores to better align the next message generation with the target behavior. Ensure that:\n\t•\tScores are modified incrementally to prevent over-adjustments, which can lead to undesired disruptions in the message quality.\n\t•\tFeatures unrelated to the target behavior are reduced in importance.\n\t•\tFeatures closely aligned with the target behavior are increased in importance.\n\t•\tThe adjustments balance improvement toward the target behavior while maintaining coherence in the generated message.\n\nExample:\n\nFeatures:\nFeatureGroup([\n    0: "Sentences citing cats",\n    1: "Cat toys",\n    2: "Sentences regarding dogs",\n])\nTarget Behavior: "Talk about cats."\nPrevious Scores: [0.1, 0.2, 0.1]\nCritique: "The sentence is more focused on dogs rather than cats."\n\nNew Scores: [0.3, 0.3, -0.2]\n\nGuidelines:\n\t1.\tUse the critique to prioritize which features to boost or suppress.\n\t2.\tAvoid overloading the weights for too many features, as excessive adjustments can lead to incoherent messages and a stronger critique from the judge.\n\t3.\tIf no critique is provided, rely on the target behavior and available features for balanced adjustments.\n\nOutput your list of new scores in the same order as the provided features.'}, {'role': 'user', 'content': '\nFeatures:\nFeatureGroup([\n   0: "Mathematical calculations and problem-solving",\n   1: "The model should solve mathematical equations",\n   2: "Prepositional phrases describing spatial relationships",\n   3: "Step-by-step mathematical problem-solving",\n   4: "Mathematical reasoning and expressions, especially in abstract mathematics and calculations"\n])\nTarget Behavior:\nBe good at solving math problems.\n\nCritique:\nThe feedback is Score: 5/10. The response demonstrates an understanding of the problem and provides a structured approach to finding the solution. However, it contains a critical flaw in the final calculation, which undermines its correctness in solving the math problem.\n\nKey Observations:\n- The breakdown and steps taken to reach the answer are well-organized. The respondent effectively explains how to calculate the time taken for one shirt and attempts to extend this to 30 shirts.\n- However, the conclusion is incorrect. The method implies a basic misunderstanding of how drying shirts works, as it does not consider that multiple shirts can dry simultaneously under the sun. Therefore, the assumption that drying time scales linearly with the number of shirts is flawed in a real-world context.\n\nSuggestions for Improvement:\n1. Re-evaluate the understanding of the problem. Since the shirts dry simultaneously under the sun, the time to dry 30 shirts would actually remain 1 hour, assuming there is enough space to lay them out under the sun.\n2. The response should clarify that the drying condition applies consistently to all shirts being dried at once.\n3. Include a summary statement at the end that clearly states the final answer based on the correct interpretation of how drying works, as well as the reasoning behind it.\n\nPositives:\n- The step-by-step approach adds clarity to the problem-solving process, showing a structured methodology.\n- The explanation of converting hours to minutes is a good detail, although it ultimately led to an incorrect conclusion., adjust the score to improve critique.\n\n'}]
